In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import math

In [ ]:
# Cplex installation on Google Colab:
!pip install cplex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 10.3 MB/s eta 0:00:00


In [ ]:
# Cyipopt installation on Google Colab:
!apt install build-essential pkg-config python3-pip python3-dev cython3 python3-numpy coinor-libipopt1v5 coinor-libipopt-dev

!pip install cyipopt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  libmumps-5.4 libmumps-dev libmumps-headers-dev libmumps-seq-5.4 libmumps-seq-dev
  libscalapack-mpi-dev libscalapack-openmpi-dev libscalapack-openmpi2.1 libscotch-6.1
  python3-setuptools python3-wheel
Suggested packages:
  cython-doc scalapack-doc python-numpy-doc python3-pytest python-setuptools-doc
The following NEW packages will be installed:
  coinor-libipopt-dev coinor-libipopt1v5 cython3 libmumps-5.4 libmumps-dev libmumps-headers-dev
  libmumps-seq-5.4 libmumps-seq-dev libscalapack-mpi-dev libscalapack-openmpi-dev
  libscalapack-openmpi2.1 libscotch-6.1 python3-numpy python3-pip python3-setuptools python3-wheel
0 

In [ ]:
import matplotlib.pyplot as plt
from scipy import sparse
import cplex
import cyipopt as ipopt

In [ ]:
# Complete the following functions
def strat_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
   x_optimal = x_init
   cash_optimal = cash_init
   return x_optimal, cash_optimal

def strat_equally_weighted(x_init, cash_init, mu, Q, cur_prices):

    return x_optimal, cash_optimal

def strat_min_variance(x_init, cash_init, mu, Q, cur_prices):

   return x_optimal, cash_optimal

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):

   return x_optimal, cash_optimal

def strat_equal_risk_contr(x_init, cash_init, mu, Q, cur_prices):

   return x_optimal, cash_optimal

def strat_lever_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):

   return x_optimal, cash_optimal

def strat_robust_optim(x_init, cash_init, mu, Q, cur_prices):

   return x_optimal, cash_optimal

# **Strategies**

***Buy and Hold***

In [ ]:
def strat_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
   x_optimal = x_init
   cash_optimal = cash_init
   return x_optimal, cash_optimal

***Equally Weighted***

In [ ]:
def strat_equally_weighted(x_init, cash_init, mu, Q, cur_prices):
    N = len(mu)  # Number of assets
    total_value = np.dot(cur_prices, x_init) + cash_init  # total portfolio value
    target_value_per_asset = total_value / N  # Target value for each asset

    target_shares = target_value_per_asset / cur_prices # target num of shares for each asset
    shares_to_trade = np.floor(target_shares) - x_init # actual num of shares to buy/sell (rounded)

    x_optimal = x_init + shares_to_trade # Update positions

    # cash account update
    cash_spent = np.sum(shares_to_trade * cur_prices)
    cash_optimal = cash_init - cash_spent

    return x_optimal, cash_optimal

***MInimum Variance***

In [ ]:
def strat_min_variance(x_init, cash_init, mu, Q, cur_prices):
    cpx = cplex.Cplex()
    cpx.objective.set_sense(cpx.objective.sense.minimize)

    n = len(mu)  # Num of assets

    c = [0.0] * n  # objective function coefficients (trying to min variance, so no linear cost)
    lb = [0.0] * n  # lower bounds
    ub = [1.0] * n  # Upper bounds


    var_names = ["w_%s" % i for i in range(n)]  # Variable names
    cpx.variables.add(obj=c, lb=lb, ub=ub, names=var_names)

    # constraint: Sum of weights = 1
    weights_sum_constraint = cplex.SparsePair(ind=var_names, val=[1.0]*n)
    cpx.linear_constraints.add(lin_expr=[weights_sum_constraint], senses="E", rhs=[1.0])

    # quadratic part of the objective function (i.e. variance)
    Qmat = [[list(range(n)), list(2*Q[k, :])] for k in range(n)]
    cpx.objective.set_quadratic(Qmat)

    cpx.parameters.qpmethod.set(4)

    cpx.set_results_stream(None)

    cpx.solve()

    # Get optimal weights
    w_minVar = np.array(cpx.solution.get_values())

    # Portfolio calcs
    total_value = np.dot(x_init, cur_prices) + cash_init
    x_optimal = np.floor((w_minVar * total_value) / cur_prices)
    optimal_portfolio_value = np.dot(x_optimal, cur_prices)
    cash_optimal = total_value - optimal_portfolio_value

    var_minVar = np.dot(w_minVar, np.dot(Q, w_minVar))
    ret_minVar = np.dot(mu, w_minVar)

    return x_optimal, cash_optimal

***Max Sharpe Ratio***

In [ ]:
import numpy as np
import cvxpy as cp

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):
    # Number of assets
    n = len(mu)
    r_rf = 0.045

    assert isinstance(x_init, np.ndarray), "x_init should be a numpy array"
    assert isinstance(mu, np.ndarray), "mu should be a numpy array"
    assert isinstance(Q, np.ndarray), "Q should be a numpy array"
    assert isinstance(cur_prices, np.ndarray), "cur_prices should be a numpy array"
    assert isinstance(cash_init, (int, float)), "cash_init should be a scalar (int or float)"

    assert x_init.ndim == 1, "x_init should be a 1-D array"
    assert mu.ndim == 1, "mu should be a 1-D array"
    assert Q.ndim == 2, "Q should be a 2-D array"
    assert cur_prices.ndim == 1, "cur_prices should be a 1-D array"

    assert x_init.shape[0] == cur_prices.shape[0], "x_init and cur_prices should have the same length"
    assert mu.shape[0] == cur_prices.shape[0], "mu and cur_prices should have the same length"
    assert Q.shape[0] == Q.shape[1], "Q should be a square matrix"
    assert Q.shape[0] == mu.shape[0], "Q should have the same dimensions as the length of mu"

    assert np.allclose(Q, Q.T), "Q should be symmetric"
    assert np.all(np.linalg.eigvals(Q) >= 0), "Q should be positive semi-definite"

    assert np.all(cur_prices > 0), "cur_prices should have positive values"
    assert np.all(x_init >= 0), "x_init should have non-negative values"

    assert np.all(mu > r_rf), "All assets should have an expected return greater than the risk-free rate"

    # Variables for optimization
    y = cp.Variable(n)
    k = cp.Variable()

    # Objective function: minimize variance of portfolio considering risk-free rate
    risk_adjusted_return = mu - r_rf
    objective = cp.Minimize(cp.quad_form(y, Q))

    # Constraints
    constraints = [
        risk_adjusted_return @ y == 1,  # Adjusted return for the portfolio factoring in risk-free rate
        cp.sum(y) == k,                # Sum of y equals k
        y >= 0                         # Weights must be non-negative
    ]

    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.CPLEX)

    if problem.status == cp.OPTIMAL:
        # Compute the optimal weights if the problem is solved optimally
        w_optimal = y.value / k.value
        cash_optimal = cash_init + np.dot(x_init, cur_prices) - np.dot(w_optimal, cur_prices)
    else:
        # If the problem is infeasible or unbounded, return the current positions and cash
        print("Problem is", problem.status, "for strat_max_Sharpe; using current positions.")
        w_optimal = np.array(x_init)  # Ensure that w_optimal is a NumPy array, not None
        cash_optimal = cash_init  # Ensure that cash_optimal is a number, not None

    # # Compute the optimal weights
    # w_optimal = y.value / k.value

    # # Compute the current portfolio value
    # portfolio_value = np.dot(cur_prices, x_init)

    # # Compute the cash after rebalancing to the optimal weights
    # cash_optimal = cash_init + portfolio_value - np.dot(w_optimal, cur_prices)

    # Return the optimal weights and cash
    return w_optimal, cash_optimal

# This function now needs to be integrated into your existing strategy framework
# and called with the appropriate parameters.

In [ ]:
import numpy as np
import cvxpy as cp

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):
    # Number of assets
    r_rf = 0.045

    valid_assets = mu > r_rf
    mu_filtered = mu[valid_assets]
    Q_filtered = Q[np.ix_(valid_assets, valid_assets)]
    cur_prices_filtered = cur_prices[valid_assets]
    x_init_filtered = x_init[valid_assets]

    # Number of filtered assets
    n_filtered = len(mu_filtered)

    # Variables for optimization
    y = cp.Variable(n_filtered)
    k = cp.Variable()

    # Objective function: minimize variance of portfolio considering risk-free rate
    risk_adjusted_return = mu_filtered - r_rf
    objective = cp.Minimize(cp.quad_form(y, Q_filtered))

    # Constraints
    constraints = [
        risk_adjusted_return @ y == 1,  # Adjusted return for the portfolio factoring in risk-free rate
        cp.sum(y) == k,                 # Sum of y equals k
        y >= 0                          # Weights must be non-negative
    ]

    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.CPLEX)

    if problem.status == cp.OPTIMAL:
        # Compute the optimal weights for filtered assets
        w_optimal_filtered = y.value / k.value
        # Extend the optimal weights to include all assets, setting weights of excluded assets to 0
        w_optimal = np.zeros_like(mu)
        w_optimal[valid_assets] = w_optimal_filtered
        # Compute the cash after rebalancing to the optimal weights
        cash_optimal = cash_init + np.dot(x_init, cur_prices) - np.dot(w_optimal, cur_prices)
    else:
        # If the problem is infeasible or unbounded, return the current positions and cash
        print("Problem is", problem.status, "for strat_max_Sharpe; using current positions.")
        w_optimal = np.array(x_init)
        cash_optimal = cash_init

    return w_optimal, cash_optimal

In [ ]:

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices, r_rf=0.045):
    valid_assets = mu > r_rf
    if not np.any(valid_assets):
        print("No assets with expected return higher than the risk-free rate; using current positions.")
        return np.array(x_init), cash_init

    mu_filtered = mu[valid_assets]
    Q_filtered = Q[np.ix_(valid_assets, valid_assets)]
    cur_prices_filtered = cur_prices[valid_assets]
    x_init_filtered = x_init[valid_assets]

    n_filtered = len(mu_filtered)
    y = cp.Variable(n_filtered)
    k = cp.Variable()

    risk_adjusted_return = mu_filtered - r_rf
    objective = cp.Minimize(cp.quad_form(y, Q_filtered))
    constraints = [
        risk_adjusted_return @ y == 1,
        cp.sum(y) == k,
        y >= 0
    ]

    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.CPLEX)

    if problem.status == cp.OPTIMAL:
        w_optimal_filtered = y.value / k.value
        w_optimal = np.zeros_like(mu)
        w_optimal[valid_assets] = w_optimal_filtered
        cash_optimal = cash_init + np.dot(x_init, cur_prices) - np.dot(w_optimal, cur_prices)
    else:
        print("Problem is", problem.status, "for strat_max_Sharpe; using current positions.")
        w_optimal = np.array(x_init)
        cash_optimal = cash_init

    return w_optimal, cash_optimal

In [ ]:
import cplex
from cplex.exceptions import CplexError
import numpy as np

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):
    r_rf = 0.025  # Annual risk-free rate for years 2020-2021
    try:
        # Calculate the number of assets
        n_assets = len(mu)

        # Current portfolio value
        portfolio_value = sum(x_init * cur_prices)

        # Initialize the CPLEX problem
        prob = cplex.Cplex()
        prob.objective.set_sense(prob.objective.sense.maximize)

        # Define variables y[i] for assets
        y_names = ['y_{0}'.format(i) for i in range(n_assets)]
        prob.variables.add(names=y_names)

        # Define variable kappa (k)
        kappa_name = 'kappa'
        prob.variables.add(names=[kappa_name], lb=[0])

        # Set the objective to maximize y.T * (mu - r_f)
        linear_obj = [(y_names[i], float(mu[i] - r_rf)) for i in range(n_assets)]
        prob.objective.set_linear(linear_obj)

        # Convert Q to a list of lists and add the quadratic part of the objective function
        Q_list = Q.tolist()
        quadratic_obj = [(i, j, float(Q_list[i][j])) for i in range(n_assets) for j in range(i, n_assets)]
        prob.objective.set_quadratic_coefficients(quadratic_obj)

        # Add the constraint that sum(mu[i] - r_f) * y[i] = 1
        linear_constraints = [cplex.SparsePair(ind=y_names, val=(mu - r_rf).tolist())]
        prob.linear_constraints.add(lin_expr=linear_constraints, senses='E', rhs=[1])

        # Add the constraint that sum(y[i]) - kappa = 0
        kappa_constraints = [cplex.SparsePair(ind=y_names + [kappa_name], val=[1.0] * n_assets + [-1.0])]
        prob.linear_constraints.add(lin_expr=kappa_constraints, senses='E', rhs=[0])

        # Solve the optimization problem
        prob.solve()

        # Retrieve the optimal y and kappa
        y_optimal = prob.solution.get_values(y_names)
        kappa_optimal = prob.solution.get_values(kappa_name)

        # Calculate the optimal w (w*) = y / kappa
        w_star = [y / kappa_optimal for y in y_optimal]

        # Given the current prices, calculate the number of shares for each asset
        x_optimal = [max(0, w * portfolio_value / price) for w, price in zip(w_star, cur_prices)]

        # Calculate cash remaining after rebalancing
        transaction_costs = sum(abs(x_opt - x_i) * p_i for x_opt, x_i, p_i in zip(x_optimal, x_init, cur_prices))
        cash_optimal = cash_init - transaction_costs + sum((x_i - x_opt) * p_i for x_opt, x_i, p_i in zip(x_optimal, x_init, cur_prices))

        return x_optimal, cash_optimal

    except CplexError as exc:
        print("CPLEX encountered an error:", exc)
        return None, None


In [ ]:
import cplex
from cplex.exceptions import CplexError
import numpy as np

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):
    r_rf = 0.025  # Annual risk-free rate for years 2020-2021
    try:
        # Calculate the number of assets
        n_assets = len(mu)

        # Current portfolio value
        portfolio_value = sum(x_init * cur_prices)

        # Initialize the CPLEX problem
        prob = cplex.Cplex()
        prob.objective.set_sense(prob.objective.sense.maximize)

        # Allow for non-convex optimization
        prob.parameters.optimalitytarget.set(2)  # 2 for local optimality

        # Define variables y[i] for assets
        y_names = ['y_{0}'.format(i) for i in range(n_assets)]
        prob.variables.add(names=y_names)

        # Define variable kappa (k)
        kappa_name = 'kappa'
        prob.variables.add(names=[kappa_name], lb=[0])

        # Set the objective to maximize y.T * (mu - r_f)
        linear_obj = [(y_names[i], float(mu[i] - r_rf)) for i in range(n_assets)]
        prob.objective.set_linear(linear_obj)

        # Convert Q to a list of lists and add the quadratic part of the objective function
        Q_list = Q.tolist()
        quadratic_obj = [(i, j, float(Q_list[i][j])) for i in range(n_assets) for j in range(i, n_assets)]
        prob.objective.set_quadratic_coefficients(quadratic_obj)

        # Add the constraint that sum(mu[i] - r_f) * y[i] = 1
        linear_constraints = [cplex.SparsePair(ind=y_names, val=(mu - r_rf).tolist())]
        prob.linear_constraints.add(lin_expr=linear_constraints, senses='E', rhs=[1])

        # Add the constraint that sum(y[i]) - kappa = 0
        kappa_constraints = [cplex.SparsePair(ind=y_names + [kappa_name], val=[1.0] * n_assets + [-1.0])]
        prob.linear_constraints.add(lin_expr=kappa_constraints, senses='E', rhs=[0])

        # Solve the optimization problem
        prob.solve()

        # Retrieve the optimal y and kappa
        y_optimal = prob.solution.get_values(y_names)
        kappa_optimal = prob.solution.get_values(kappa_name)

        # Calculate the optimal w (w*) = y / kappa
        w_star = [y / kappa_optimal for y in y_optimal]

        # Given the current prices, calculate the number of shares for each asset
        x_optimal = [max(0, w * portfolio_value / price) for w, price in zip(w_star, cur_prices)]

        # Calculate cash remaining after rebalancing
        transaction_costs = sum(abs(x_opt - x_i) * p_i for x_opt, x_i, p_i in zip(x_optimal, x_init, cur_prices))
        cash_optimal = cash_init - transaction_costs + sum((x_i - x_opt) * p_i for x_opt, x_i, p_i in zip(x_optimal, x_init, cur_prices))

        return x_optimal, cash_optimal

    except CplexError as exc:
        print("CPLEX encountered an error:", exc)
        return None, None


In [ ]:
import cplex
from cplex.exceptions import CplexError
import numpy as np

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):
    r_rf = 0.025  # Annual risk-free rate for years 2020-2021
    try:
        # Calculate the number of assets
        n_assets = len(mu)

        # Current portfolio value
        portfolio_value = sum(x_init * cur_prices)

        # Initialize the CPLEX problem
        prob = cplex.Cplex()
        prob.objective.set_sense(prob.objective.sense.maximize)

        # Set parameters for CPLEX to find a solution even if the problem is non-convex
        prob.parameters.threads.set(1)  # Use one thread for consistency in repeated runs
        prob.parameters.optimalitytarget.set(2)  # Set optimality target to 2 (local optimum) or 3 (global optimum)

        # Define variables y[i] for assets
        y_names = ['y_{0}'.format(i) for i in range(n_assets)]
        prob.variables.add(names=y_names)

        # Define variable kappa (k)
        kappa_name = 'kappa'
        prob.variables.add(names=[kappa_name], lb=[0])

        # Set the objective to maximize y.T * (mu - r_f)
        linear_obj = [(y_names[i], float(mu[i] - r_rf)) for i in range(n_assets)]
        prob.objective.set_linear(linear_obj)

        # Convert Q to a list of lists and add the quadratic part of the objective function
        Q_list = Q.tolist()
        quadratic_obj = [(i, j, float(Q_list[i][j])) for i in range(n_assets) for j in range(i, n_assets)]
        prob.objective.set_quadratic_coefficients(quadratic_obj)

        # Add the constraint that sum(mu[i] - r_f) * y[i] = 1
        linear_constraints = [cplex.SparsePair(ind=y_names, val=(mu - r_rf).tolist())]
        prob.linear_constraints.add(lin_expr=linear_constraints, senses='E', rhs=[1])

        # Add the constraint that sum(y[i]) - kappa = 0
        kappa_constraints = [cplex.SparsePair(ind=y_names + [kappa_name], val=[1.0] * n_assets + [-1.0])]
        prob.linear_constraints.add(lin_expr=kappa_constraints, senses='E', rhs=[0])

        # Solve the optimization problem
        prob.solve()

        # Retrieve the optimal y and kappa
        y_optimal = prob.solution.get_values(y_names)
        kappa_optimal = prob.solution.get_values(kappa_name)

        # Calculate the optimal w (w*) = y / kappa
        w_star = [y / kappa_optimal if kappa_optimal != 0 else 0 for y in y_optimal]

        # Given the current prices, calculate the number of shares for each asset
        x_optimal = [max(0, w * portfolio_value / price) for w, price in zip(w_star, cur_prices)]

        # Calculate cash remaining after rebalancing
        transaction_costs = sum(abs(x_opt - x_i) * p_i for x_opt, x_i, p_i in zip(x_optimal, x_init, cur_prices))
        cash_optimal = cash_init - transaction_costs + sum((x_i - x_opt) * p_i for x_opt, x_i, p_i in zip(x_optimal, x_init, cur_prices))

        return x_optimal, cash_optimal

    except CplexError as exc:
        print("CPLEX encountered an error:", exc)
        return None, None


***Equal Risk Contribution***

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Risk Contribution calculation
def risk_contribution(w, Q):
    sigma = np.sqrt(w @ Q @ w)
    # Marginal risk contribution
    MRC = Q @ w
    # Risk contribution per asset
    RC = w * MRC / sigma
    return RC

# Objective function
def objective(w, Q):
    RC = risk_contribution(w, Q)
    # We subtract the average because we want to minimize the difference of each RC from the average
    avg_RC = np.mean(RC)
    sum_squared_deviation = np.sum((RC - avg_RC)**2)
    return sum_squared_deviation

# Constraint: Sum of weights = 1
def constraint(w):
    return np.sum(w) - 1

# Optimization function
def strat_equal_risk_contr(x_init, cash_init, mu, Q, cur_prices):
    n = len(x_init)
    w0 = x_init / np.dot(x_init, cur_prices) # Initial weights
    constraints = ({'type': 'eq', 'fun': constraint})
    bounds = [(0, 1)] * n # No short selling
    options = {'disp': False, 'maxiter': 1000, 'ftol': 1e-10}

    # Minimize the objective function
    res = minimize(fun=objective, x0=w0, args=(Q,), method='SLSQP',
                   bounds=bounds, constraints=constraints, options=options)

    if not res.success:
        raise ValueError("Optimization did not converge")

    w_optimal = res.x
    total_value = np.dot(x_init, cur_prices) + cash_init
    x_optimal = w_optimal * total_value / cur_prices # Transform weights back to number of shares
    cash_optimal = cash_init - np.sum((x_optimal - x_init) * cur_prices) # Cash after buying/selling shares

    return x_optimal, cash_optimal

# Example usage:
# x_optimal, cash_optimal = strat_equal_risk_contr(x_init, cash_init, mu, Q, cur_prices)

# Implementation

In [ ]:
# Input file
input_file_prices = 'Daily_closing_prices2020_2021.csv'
#input_file_prices = 'Daily_closing_prices2022.csv'
#input_file_prices = 'Daily_closing_prices2008_2009.csv'
# input_file_prices = 'Daily_closing_prices.csv'

In [ ]:
# Read data into a dataframe
df = pd.read_csv(input_file_prices)

In [ ]:
# Convert dates into array [year month day]
def convert_date_to_array(datestr):
    """
    Loads daily closing prices from a CSV file into a Pandas DataFrame.
    Converts string dates to a list of integers [year, month, day].
    """
    temp = [int(x) for x in datestr.split('/')]
    return [temp[-1], temp[0], temp[1]]

dates_array = np.array(list(df['Date'].apply(convert_date_to_array)))
data_prices = df.iloc[:, 1:].to_numpy()
dates = np.array(df['Date'])
# Find the number of trading days in Nov-Dec 2019 and
# compute expected return and covariance matrix for period 1
day_ind_start0 = 0
day_ind_end0 = len(np.where(dates_array[:,0]==2019)[0])   # for 2020-2021 csv

#day_ind_end0 = len(np.where(dates_array[:,0]==2007)[0])   # for 2008-2009 csv
#day_ind_end0 = len(np.where(dates_array[:,0]==2021)[0])   # for 2022 csv

cur_returns0 = data_prices[day_ind_start0+1:day_ind_end0,:] / data_prices[day_ind_start0:day_ind_end0-1,:] - 1
mu = np.mean(cur_returns0, axis = 0)
Q = np.cov(cur_returns0.T)

# Remove datapoints for year 2019
data_prices = data_prices[day_ind_end0:,:]
dates_array = dates_array[day_ind_end0:,:]
dates = dates[day_ind_end0:]

''' Sets initial portfolio positions (init_positions) and calculates the initial portfolio value (init_value).'''
# Initial positions in the portfolio
init_positions = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 902, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17500])
# Initial value of the portfolio
init_value = np.dot(data_prices[0,:], init_positions)
print('\nInitial portfolio value = $ {}\n'.format(round(init_value, 2)))

# Initial portfolio weights
'''Calculates initial portfolio weights (w_init) based on the initial value and the current prices.'''
w_init = (data_prices[0,:] * init_positions) / init_value


'''
Defines the number of periods, assets, and trading days for the strategies.
Sets different risk-free rates for different historical periods, though only one
will be used depending on the data year.
'''
# Number of periods, assets, trading days
N_periods = 6*len(np.unique(dates_array[:,0])) # 6 periods per year
N = len(df.columns)-1
N_days = len(dates)

# Annual risk-free rate for years 2020-2021 is 2.5%
r_rf = 0.025
# Annual risk-free rate for years 2008-2009 is 4.5%
r_rf2008_2009 = 0.045
# Annual risk-free rate for year 2022 is 3.75%
r_rf2022 = 0.0375

# Number of strategies
strategy_functions = ['strat_buy_and_hold', 'strat_equally_weighted', 'strat_min_variance', 'strat_max_Sharpe', 'strat_equal_risk_contr']#, 'strat_lever_max_Sharpe', 'strat_robust_optim']
strategy_names     = ['Buy and Hold', 'Equally Weighted Portfolio', 'Mininum Variance Portfolio', 'Maximum Sharpe Ratio Portfolio', 'Equal Risk Contributions Portfolio'] #, , 'Leveraged Max Sharpe Ratio Portfolio', 'Robust Optimization Portfolio']


# strategy_functions = ['strat_max_Sharpe'] #'] #,, 'strat_equal_risk_contr', 'strat_lever_max_Sharpe', 'strat_robust_optim']
# strategy_names     = [ 'Maximum Sharpe Ratio Portfolio'] #, 'Mininum Variance Portfolio', 'Maximum Sharpe Ratio Portfolio', 'Equal Risk Contributions Portfolio', 'Leveraged Max Sharpe Ratio Portfolio', 'Robust Optimization Portfolio']


#N_strat = 1  # comment this in your code
N_strat = len(strategy_functions) - 1  # uncomment this in your code

# fh_array = [strat_buy_and_hold, strat_equally_weighted, strat_min_variance, strat_max_Sharpe, strat_equal_risk_contr, strat_lever_max_Sharpe, strat_robust_optim]
fh_array = [strat_buy_and_hold, strat_equally_weighted, strat_min_variance, strat_max_Sharpe, strat_equal_risk_contr]
'''
Initializes arrays to store portfolio values (portf_value), asset positions (x),
and cash holdings (cash).
'''
portf_value = [0] * N_strat
x = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
cash = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
for period in range(1, N_periods+1):
   # Compute current year and month, first and last day of the period

   # Depending on what data/csv (i.e time period) uncomment code
   if dates_array[0, 0] == 20:
       cur_year  = 20 + math.floor(period/7)
   else:
       cur_year  = 2020 + math.floor(period/7)

   # example for 2008-2009 data
   #if dates_array[0, 0] == 8:
   #    cur_year  = 8 + math.floor(period/7)
   #else:
   #    cur_year  = 2008 + math.floor(period/7)


   cur_month = 2*((period-1)%6) + 1

   ''' debugging - deleter later'''

   start_indices = [i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month)) if val]
   if start_indices:
       day_ind_start = min(start_indices)
   else:
       print(f"No data available for {cur_month}/{cur_year}")
       continue  # Skip to the next period

   end_indices = [i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month+1)) if val]
   if end_indices:
       day_ind_end = max(end_indices)
   else:
       print(f"No data available for {cur_month+1}/{cur_year}")
       continue  # Skip to the next period


   ''' ****************************************** '''

   day_ind_start = min([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month)) if val])
   day_ind_end = max([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month+1)) if val])
   print('\nPeriod {0}: start date {1}, end date {2}'.format(period, dates[day_ind_start], dates[day_ind_end]))


   cur_prices = data_prices[day_ind_start,:]

   # Execute portfolio selection strategies
   for strategy  in range(N_strat):

      # Get current portfolio positions
      if period == 1:
         curr_positions = init_positions
         curr_cash = 0
         portf_value[strategy] = np.zeros((N_days, 1))
      else:
         curr_positions = x[strategy, period-2]
         curr_cash = cash[strategy, period-2]

      # Compute strategy
      # x[strategy, period-1], cash[strategy, period-1] = fh_array[strategy](curr_positions, curr_cash, mu, Q, cur_prices)

      result = fh_array[strategy](curr_positions, curr_cash, mu, Q, cur_prices)

      if result[0] is not None and result[1] is not None:
          x[strategy, period-1], cash[strategy, period-1] = result
      else:
          # If result is None, use the current positions and cash for this strategy
          x[strategy, period-1] = np.array(curr_positions)
          cash[strategy, period-1] = curr_cash
          continue  # Skip the rest of the current iteration and move to the next strategy

      # Now you can safely calculate position changes as both arrays are guaranteed to be not None
      new_positions = x[strategy, period-1]
      position_changes = new_positions - curr_positions

      # Verify that strategy is feasible (you have enough budget to re-balance portfolio)
      # Check that cash account is >= 0
      # Check that we can buy new portfolio subject to transaction costs

      ###################### Insert your code here ############################
      # transaction cost rate
      trans_cost_rate = 0.005

      # cost of buying new portfolio
      new_positions = x[strategy, period-1]
      position_changes = new_positions - curr_positions
      buying_cost = np.sum(position_changes[position_changes > 0] * cur_prices[position_changes > 0]) * trans_cost_rate

      # revenue from selling part of portfolio
      selling_revenue = np.abs(np.sum(position_changes[position_changes < 0] * cur_prices[position_changes < 0]) * trans_cost_rate)

      # cash account update for transaction costs
      curr_cash -= buying_cost
      curr_cash += selling_revenue

      # check if enough budget for rebalancing including transaction costs
      if curr_cash < 0:
          print(f"Strategy '{strategy_names[strategy]}' is not feasible due to insufficient budget for rebalancing.")
      else:
          # cash account update
          cash[strategy, period-1] = curr_cash

          # make sure acount is non-negative
          if cash[strategy, period-1] < 0:
              print(f"Cash account for strategy '{strategy_names[strategy]}' is negative, which is not allowed.")

      #########################################################################

      # Compute portfolio value
      p_values = np.dot(data_prices[day_ind_start:day_ind_end+1,:], x[strategy, period-1]) + cash[strategy, period-1]
      portf_value[strategy][day_ind_start:day_ind_end+1] = np.reshape(p_values, (p_values.size,1))
      print('  Strategy "{0}", value begin = $ {1:.2f}, value end = $ {2:.2f}'.format( strategy_names[strategy],
             portf_value[strategy][day_ind_start][0], portf_value[strategy][day_ind_end][0]))


   # Compute expected returns and covariances for the next period
   cur_returns = data_prices[day_ind_start+1:day_ind_end+1,:] / data_prices[day_ind_start:day_ind_end,:] - 1
   mu = np.mean(cur_returns, axis = 0)
   Q = np.cov(cur_returns.T)



Initial portfolio value = $ 1000012.93


Period 1: start date 01/02/2020, end date 02/28/2020
  Strategy "Buy and Hold", value begin = $ 1000012.93, value end = $ 893956.75
  Strategy "Equally Weighted Portfolio", value begin = $ 997991.06, value end = $ 899257.01
  Strategy "Mininum Variance Portfolio", value begin = $ 998969.09, value end = $ 921804.97
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_OptimalityTarget                        2
Number of nonzeros in lower triangle of Q = 190
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 20
  Integer space required    = 20
  Total non-zeros in factor = 210
  Total FP ops to factor    = 2870
Note: Q in objective is not positive semi-definite.
Row 'c1' infeasible, all entries at implied bounds.
Presolve ti

CPLEX Error  1217: No solution exists.


CPLEX encountered an error: CPLEX Error  1217: No solution exists.

Period 2: start date 03/02/2020, end date 04/30/2020
  Strategy "Buy and Hold", value begin = $ 945076.08, value end = $ 949228.39
  Strategy "Equally Weighted Portfolio", value begin = $ 936521.01, value end = $ 867452.45
  Strategy "Mininum Variance Portfolio", value begin = $ 967811.16, value end = $ 861468.97
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_OptimalityTarget                        2
Number of nonzeros in lower triangle of Q = 190
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 20
  Integer space required    = 20
  Total non-zeros in factor = 210
  Total FP ops to factor    = 2870
Note: Q in objective is not positive semi-definite.
Row 'c1' infeasible, all entries at im

CPLEX Error  1217: No solution exists.


CPLEX encountered an error: CPLEX Error  1217: No solution exists.

Period 3: start date 05/01/2020, end date 06/30/2020
  Strategy "Buy and Hold", value begin = $ 937916.81, value end = $ 913415.30
  Strategy "Equally Weighted Portfolio", value begin = $ 834381.95, value end = $ 937940.69
  Strategy "Mininum Variance Portfolio", value begin = $ 838602.34, value end = $ 866440.66
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_OptimalityTarget                        2
Number of nonzeros in lower triangle of Q = 190
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 20
  Integer space required    = 20
  Total non-zeros in factor = 210
  Total FP ops to factor    = 2870
Note: Q in objective is not positive semi-definite.
Row 'c1' infeasible, all entries at im

...

CPLEX encountered an error: CPLEX Error  1217: No solution exists.

Period 11: start date 09/01/2021, end date 10/29/2021
  Strategy "Buy and Hold", value begin = $ 974520.08, value end = $ 949068.41
  Strategy "Equally Weighted Portfolio", value begin = $ 1497615.83, value end = $ 1547129.61
  Strategy "Mininum Variance Portfolio", value begin = $ 1114549.74, value end = $ 1089859.81
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_OptimalityTarget                        2
Number of nonzeros in lower triangle of Q = 190
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.01 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 20
  Integer space required    = 20
  Total non-zeros in factor = 210
  Total FP ops to factor    = 2870
Note: Q in objective is not positive semi-definite.
Row 'c1' infeasible, all entries 

CPLEX Error  1217: No solution exists.


CPLEX encountered an error: CPLEX Error  1217: No solution exists.

Period 12: start date 11/01/2021, end date 12/31/2021
  Strategy "Buy and Hold", value begin = $ 951350.41, value end = $ 932471.35
  Strategy "Equally Weighted Portfolio", value begin = $ 1565233.79, value end = $ 1626424.61
  Strategy "Mininum Variance Portfolio", value begin = $ 1088887.86, value end = $ 1082838.29
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_OptimalityTarget                        2
Number of nonzeros in lower triangle of Q = 190
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 20
  Integer space required    = 20
  Total non-zeros in factor = 210
  Total FP ops to factor    = 2870
Note: Q in objective is not positive semi-definite.
Row 'c1' infeasible, all entries 

CPLEX Error  1217: No solution exists.


CPLEX encountered an error: CPLEX Error  1217: No solution exists.


*EQPT*

In [ ]:
import cplex
import numpy as np

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):
    # Number of assets
    n_assets = len(mu)

    # Create a CPLEX problem
    prob = cplex.Cplex()

    # Set the problem to maximization
    prob.objective.set_sense(prob.objective.sense.maximize)

    # Add variables y (asset weights) and kappa
    y_names = ['y_' + str(i) for i in range(n_assets)]
    kappa_name = ['kappa']

    # Set the objective to minimize y^T * Q * y
    # Here we use the fact that maximizing Sharpe's ratio is equivalent to minimizing y^T * Q * y
    # for a transformed variable y, which represents the weights divided by kappa
    prob.variables.add(obj=[0.0] * n_assets + [1.0],  # Objective function (only kappa has a coefficient in the quadratic part)
                       names=y_names + kappa_name)

    threshold = 1e-10

    # Step 1: Round small negative numbers to zero
    Q = np.where((Q < 0) & (Q > -threshold), 0, Q)

    # Step 2: Ensure the matrix is positive semi-definite
    eigenvalues, eigenvectors = np.linalg.eigh(Q)
    # Add a small number to negative eigenvalues to make them nonnegative
    positive_eigenvalues = np.where(eigenvalues < 0, 0, eigenvalues)
    # Reconstruct the positive semi-definite covariance matrix
    Q = eigenvectors @ np.diag(positive_eigenvalues) @ eigenvectors.T



    # Add the quadratic part of the objective
    qmat = []
    for i in range(n_assets):
        for j in range(i, n_assets):  # only need to specify the upper triangle due to symmetry
            if Q[i, j] != 0:  # Only add non-zero entries to the matrix
                qmat.append((i, j, 2 * Q[i, j]))  # CPLEX uses zero-based indexing


    prob.objective.set_quadratic(qmat)

    # Linear constraint: Sum(y_i * (mu_i - r_f)) = 1
    lin_expr = [[y_names, list(mu - cash_init)]]
    prob.linear_constraints.add(lin_expr=lin_expr,
                                senses='E',
                                rhs=[1.0])

    # Budget constraint: Sum(y_i * cur_prices) = kappa
    lin_expr = [[y_names + kappa_name, list(cur_prices) + [-1.0]]]
    prob.linear_constraints.add(lin_expr=lin_expr,
                                senses='E',
                                rhs=[0.0])

    # Solve the problem
    prob.solve()

    # Get the optimal solution
    y_optimal = prob.solution.get_values(y_names)
    kappa_optimal = prob.solution.get_values(kappa_name)[0]
    w_optimal = np.array(y_optimal) / kappa_optimal

    # Rescale the weights to get the actual weights for the portfolio
    # and calculate the optimal cash position
    total_value = np.dot(cur_prices, x_init) + cash_init
    portfolio_weights = total_value * w_optimal
    cash_optimal = cash_init - np.dot((portfolio_weights - np.array(x_init) * cur_prices), cur_prices)

    return portfolio_weights, cash_optimal

In [ ]:
# Convert dates into array [year month day]
def convert_date_to_array(datestr):
    """
    Loads daily closing prices from a CSV file into a Pandas DataFrame.
    Converts string dates to a list of integers [year, month, day].
    """
    temp = [int(x) for x in datestr.split('/')]
    return [temp[-1], temp[0], temp[1]]

dates_array = np.array(list(df['Date'].apply(convert_date_to_array)))
data_prices = df.iloc[:, 1:].to_numpy()
dates = np.array(df['Date'])
# Find the number of trading days in Nov-Dec 2019 and
# compute expected return and covariance matrix for period 1
day_ind_start0 = 0
day_ind_end0 = len(np.where(dates_array[:,0]==2019)[0])   # for 2020-2021 csv

cur_returns0 = data_prices[day_ind_start0+1:day_ind_end0,:] / data_prices[day_ind_start0:day_ind_end0-1,:] - 1
mu = np.mean(cur_returns0, axis = 0)
Q = np.cov(cur_returns0.T)

# Remove datapoints for year 2019
data_prices = data_prices[day_ind_end0:,:]
dates_array = dates_array[day_ind_end0:,:]
dates = dates[day_ind_end0:]

''' Sets initial portfolio positions (init_positions) and calculates the initial portfolio value (init_value).'''
# Initial positions in the portfolio
init_positions = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 902, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17500])
# Initial value of the portfolio
init_value = np.dot(data_prices[0,:], init_positions)
print('\nInitial portfolio value = $ {}\n'.format(round(init_value, 2)))

# Initial portfolio weights
'''Calculates initial portfolio weights (w_init) based on the initial value and the current prices.'''
w_init = (data_prices[0,:] * init_positions) / init_value


'''
Defines the number of periods, assets, and trading days for the strategies.
Sets different risk-free rates for different historical periods, though only one
will be used depending on the data year.
'''
# Number of periods, assets, trading days
N_periods = 6*len(np.unique(dates_array[:,0])) # 6 periods per year
N = len(df.columns)-1
N_days = len(dates)

# Annual risk-free rate for years 2020-2021 is 2.5%
r_rf = 0.025




strategy_functions = ['strat_max_Sharpe']
strategy_names     = [ 'Maximum Sharpe Ratio Portfolio']


N_strat = 1

fh_array = [strat_max_Sharpe]

'''
Initializes arrays to store portfolio values (portf_value), asset positions (x),
and cash holdings (cash).
'''
portf_value = [0] * N_strat
x = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
cash = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
for period in range(1, N_periods+1):
   # Compute current year and month, first and last day of the period

   # Depending on what data/csv (i.e time period) uncomment code
   if dates_array[0, 0] == 20:
       cur_year  = 20 + math.floor(period/7)
   else:
       cur_year  = 2020 + math.floor(period/7)

   # example for 2008-2009 data
   #if dates_array[0, 0] == 8:
   #    cur_year  = 8 + math.floor(period/7)
   #else:
   #    cur_year  = 2008 + math.floor(period/7)

   cur_month = 2*((period-1)%6) + 1
   day_ind_start = min([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month)) if val])
   day_ind_end = max([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month+1)) if val])
   print('\nPeriod {0}: start date {1}, end date {2}'.format(period, dates[day_ind_start], dates[day_ind_end]))

   # Prices for the current day
   cur_prices = data_prices[day_ind_start,:]

   # Execute portfolio selection strategies
   for strategy  in range(N_strat):

      # Get current portfolio positions
      if period == 1:
         curr_positions = init_positions
         curr_cash = 0
         portf_value[strategy] = np.zeros((N_days, 1))
      else:
         curr_positions = x[strategy, period-2]
         curr_cash = cash[strategy, period-2]

      # Compute strategy
      x[strategy, period-1], cash[strategy, period-1] = fh_array[strategy](curr_positions, curr_cash, mu, Q, cur_prices)

      # Verify that strategy is feasible (you have enough budget to re-balance portfolio)
      # Check that cash account is >= 0
      # Check that we can buy new portfolio subject to transaction costs

      ###################### Insert your code here ############################
      # transaction cost rate
      trans_cost_rate = 0.005

      # cost of buying new portfolio
      new_positions = x[strategy, period-1]
      position_changes = new_positions - curr_positions
      buying_cost = np.sum(position_changes[position_changes > 0] * cur_prices[position_changes > 0]) * trans_cost_rate

      # revenue from selling part of portfolio
      selling_revenue = np.abs(np.sum(position_changes[position_changes < 0] * cur_prices[position_changes < 0]) * trans_cost_rate)

      # cash account update for transaction costs
      curr_cash -= buying_cost
      curr_cash += selling_revenue

      # check if enough budget for rebalancing including transaction costs
      if curr_cash < 0:
          print(f"Strategy '{strategy_names[strategy]}' is not feasible due to insufficient budget for rebalancing.")
      else:
          # cash account update
          cash[strategy, period-1] = curr_cash

          # make sure acount is non-negative
          if cash[strategy, period-1] < 0:
              print(f"Cash account for strategy '{strategy_names[strategy]}' is negative, which is not allowed.")



      #########################################################################

      # Compute portfolio value
      p_values = np.dot(data_prices[day_ind_start:day_ind_end+1,:], x[strategy, period-1]) + cash[strategy, period-1]
      portf_value[strategy][day_ind_start:day_ind_end+1] = np.reshape(p_values, (p_values.size,1))
      print('  Strategy "{0}", value begin = $ {1:.2f}, value end = $ {2:.2f}'.format( strategy_names[strategy],
             portf_value[strategy][day_ind_start][0], portf_value[strategy][day_ind_end][0]))


   # Compute expected returns and covariances for the next period
   cur_returns = data_prices[day_ind_start+1:day_ind_end+1,:] / data_prices[day_ind_start:day_ind_end,:] - 1
   mu = np.mean(cur_returns, axis = 0)
   Q = np.cov(cur_returns.T)



Initial portfolio value = $ 1000012.93


Period 1: start date 01/02/2020, end date 02/28/2020


TypeError: object of type 'int' has no len()

In [ ]:
import numpy as np
print(np.shape(Q))
print(Q)

(20, 20)
[[ 8.80589720e-05  4.35356375e-05  3.16851552e-05  3.52983492e-05
   2.05595534e-05  4.51542633e-05  4.33796637e-05  2.97239213e-05
   2.92407554e-05  3.67373121e-05  1.31192392e-05  9.25123051e-06
   4.04349495e-05  2.96182268e-05  4.35708374e-05  2.43491561e-05
   4.53718434e-05  5.35313549e-06  1.82940587e-05 -8.30364570e-08]
 [ 4.35356375e-05  4.60269565e-04  3.66889991e-05  8.58213154e-05
   1.08103875e-04  8.66676009e-05  7.98606731e-05  7.23501019e-05
   3.93783268e-05  2.46020302e-05  5.65935063e-05  5.28999826e-05
   5.60360523e-05  8.14474656e-05  1.19109125e-04  4.13534977e-05
   1.80194562e-04  1.49857424e-05 -1.97660055e-05  1.18016759e-05]
 [ 3.16851552e-05  3.66889991e-05  1.03954973e-04  3.03723943e-05
   2.45410155e-05  4.37638686e-05  3.38730459e-05  2.87481939e-05
   2.25053378e-05  3.10156990e-05 -3.58190397e-06  1.93921411e-05
   2.96930669e-05  2.70226855e-05  3.79802863e-05  2.42612785e-05
   7.17711085e-05  1.25115807e-05  1.96272512e-06 -3.36018910e-06